In [1]:
'''
yes24 review crawler
20 reviews for each book
for book that has reviews more than 20
'''

'\nyes24 review crawler\n20 reviews for each book\nfor book that has reviews more than 20\n'

In [2]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen


In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

In [4]:
#책에 관련된 데이터(책제목, 작가, isbn, 리뷰)를 수집
#특히 리뷰를 수집할 때는 returnReviewText 함수를 사용
def crawlingBookdata():
        
    try:
        #title
        title = driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2').text
        
        #author
        author = driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a:nth-child(1)').text
        
        #isbn13
        isbnBlock = driver.find_element_by_css_selector('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td')
        if isbnBlock:
            print('isbn found')
            isbn13 = isbnBlock.text
        
        print(title,author,isbn13)
        
        #리뷰 데이터(review) 수집 
        reviewList=[]
        reviewPageNum = 4
        
        for i in range(4):
            print('review {} page'.format(i+1))
            returnReviewtext(reviewList)
            nextReview = driver.find_element_by_css_selector(f'#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child({reviewPageNum})')
            if nextReview:
                nextReviewClick = ActionChains(driver).move_to_element(nextReview).click()
                nextReviewClick.perform()
                time.sleep(10)
                print('successfully clicked next page')
                reviewPageNum = reviewPageNum + 1
    
        return [title,author,isbn13,' '.join(reviewList)]
    
    except Exception as e:
        print(e)

In [5]:
def returnReviewtext(reviewList):
    
    reviewBlock = driver.find_element_by_id('infoset_reviewContentList')
    if reviewBlock:
        print('reviewBlock found')

    childNum = 3
    while(childNum<5):
        try:
            time.sleep(5)
            eachReview = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.crop > a')
            print('each review found')

            
            #더보기 누르고
            btnClick = ActionChains(driver).move_to_element(eachReview).click()
            btnClick.perform()
            print('successfully clicked btn')
            time.sleep(5)
            

            #내용가져오고
            reviewTxt = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.origin > div.review_cont')
            reviewList.append( reviewTxt.text )
            
            #스크롤 조금 내리고
            driver.execute_script("window.scrollTo(0, window.scrollY + 1000);")
            

            childNum = childNum + 1
        except Exception as e:
            print(e)
            break
            
        
    return reviewList

In [6]:
def findBookpage():
    
    bookNum=0
    while(True):
        try:
            oddNum = 2*bookNum + 1
            
            whichBook = driver.find_element_by_css_selector(f'#category_layout > tbody > tr:nth-child({oddNum}) > td.goodsTxtInfo > p.review > a')
            
            if whichBook:
                print(f'{bookNum+1} book found')

                #check how many reviews
                reviewCnt = int(whichBook.text[:-1])
                print('review : ', reviewCnt)

                actions = ActionChains(driver).move_to_element(whichBook).click()
                actions.perform()
                
                bookNum=bookNum+1
                
                dataList.append(crawlingBookdata())
                print('append datalist')

            driver.back()
        except Exception as e:
            print(e)
            break

In [7]:
options = Options()
options.add_argument("headless")
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
#driver = webdriver.Chrome(options=options, executable_path='chrome driver 경로를 입력해주세요')
driver = webdriver.Chrome(options=options, executable_path='C:/dev_python/Webdriver/chromedriver.exe')

global dataList
dataList = list()

pageNum=1

try:    
    while(pageNum<10):
        
        print('pageNum : ', pageNum)
        
        #open yes24 page
        driver.get(f'http://www.yes24.com/24/category/bestseller?CategoryNumber=001001046&sumgb=06&PageNumber={pageNum}')
        
        findBookpage()
        
        pageNum = pageNum + 1
        
except Exception as e:
    print(e)

pageNum :  1
1 book found
review :  51
isbn found
달러구트 꿈 백화점 2 이미예 9791165343729
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
2 book found
review :  432
isbn found
달러구트 꿈 백화점 이미예 9791165341909
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review foun

each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
3 book found
review :  369
isbn found
[예스리커버] 우리가 빛의 속도로 갈 수 없다면 김초엽 9791190090018
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each re

each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
14 book found
review :  12
isbn found
애린 왕자 최현애 9791197182204
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
Message: e

each review found
successfully clicked btn
successfully clicked next page
append datalist
5 book found
review :  35
isbn found
구의 증명 최진영 9788956608556
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
6 book found
review :  35
isbn found
프로젝트 헤일메리 앤디 위어 9788925588735
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewB

each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
16 book found
review :  34
isbn found
루팡의 딸 요코제키 다이 9788998274412
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
append datalist
17 book found
review :  35
isbn found
망원동 브라더스 김호연 9788997962129
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
succes

KeyboardInterrupt: 

In [8]:
print(dataList)

[['달러구트 꿈 백화점 2', '이미예', '9791165343729', '달러구트 꿈백화점 1권을 읽으신 분들이라면 2권을 손꼽아 기다렸을 것이다. 나 또한 2권에서는 어떤 일들이 펼쳐질지 궁금했다. 누구나 꿈을 꾼다. 좋은 꿈을 꾸었다며 복권을 사는 사람들이 있고 꿈자리가 사납다는 말도 하고 악몽으로 식은땀을 흘릴 때도 있다. 이렇게 다양한 꿈들을 선택할 수 있다면 어떨까. 책에서도 꿈 일기를 쓰는 내용이 나오는데 어릴 때 꿈 일기를 쓴 적이 있다. 간혹 공상과학 영화에서나 볼 수 있는 내용들을 잊고 싶지 않아 기록을 했다. 그 꿈이 현실로 이루어질 수 없다는 것을 알지만 이루어지길 바라는 마음을 갖기도 했다. 우리들은 꿈이라는 것을 알지만 꿈에서 깨어나기 싫고 어떨 때는 꿈에서 빨리 깨어나고 싶은 마음이 생긴다. 다양한 꿈의 이야기처럼 꿈같은 일들이 펼쳐지는 책 속으로 떠나보자.\n      2권은 페니가 달러구트 꿈백화점 1층 프런트에서 일한 지 1년이 지난 후에 시작이 된다. 입사 1년이 되면 \'컴퍼니 구역\'에 들어갈 수 있는 출입증을 받을 수 있다. 그렇게 바라던 출ㄹ입증을 받고 국가에서 인정하는 \'꿈 산업 종사자\'가 되어 기쁘다, 하지만 선배들은 민원 관리국에 가는 일이 즐겁지만은 않다고 말한다. 어떤 일이든 민원이 있다는 것을 알지만 크게 문제가 되지 않는다고 생각했다. 두려움보다는 설렘을 안고 \'컴퍼니 구역\'에 들어가는 페니에게는 어떤 일들이 펼져질까.\n  "페니, 우리가 벌어들인 돈은 손님들의 귀중한 감정과 맞바꾼 것이니까 이 무게를 잊지 않도록 해야 한다." - p.44\n      2권의 부제는 \'단골손님을 찾습니다\'이다. 부제가 말해주듯 2권에서는 단골손님이었던 사람들이 꿈백화점을 찾지 않는 이유가 무엇이며 그것을 하나씩 해결해가는 페니의 모습이 담겨 있다. 처음으로 맡은 일은 792번 단골손님의 민원등급이 3단계인 민원이다. \'꿈꾸는 자체가 고통스러운 수준\'이라는 높은 단계의 민원을 페니는 잘 해결할 수 있을까. 

In [12]:
while(True):
    dataList.remove(None)

ValueError: list.remove(x): x not in list

In [13]:
print(len(dataList))

47


In [14]:
df = pd.DataFrame(dataList, columns=['title', 'author','isbn13','review'])

In [17]:
df2=pd.read_csv('yes24Reviews.csv')

In [18]:
print(df2)

                   title    author         isbn13  \
0       있으려나 서점 (여름 에디션)  요시타케 신스케  9788934981237   
1                  회색 인간       김동식  9791196222628   
2                  칼의 노래        김훈  9788954623360   
3              한국고전소설 40       권정현  9788965820499   
4               기타기타 사건부   미야베 미유키  9791191253313   
..                   ...       ...            ...   
87                  동물농장     조지 오웰  9788932909707   
88  사랑하라 한번도 상처받지 않은 것처럼       류시화  9788995501474   
89                   구운몽       김만중  9788937460722   
90                  파리대왕    윌리엄 골딩  9788937460197   
91               삶의 한가운데    루이제 린저  9788937460289   

                                               review  
0   요시타케 신스케는 참 생각이 기발한 작가인 것 같아요. 딸아이가 학교에서 요시타케 ...  
1   지인의 추천으로 회색인간을 구입했다. 표지가 조금 기괴해서(발만 있어서) 특이하다 ...  
2   나는 죽음을 죽음으로써 각오할 수는 없었다. 나는 각오되지 않는 죽음이 두려웠다.\...  
3   고입 아들을 위해 준비한 책. 학교 과제물 감상문 교재로도 채택되었다. 고등국어 예...  
4   주인공 기타이치는 버려졌는지 헤어졌는지 모르는 미아로 오캇피키센키치 대장의 눈에 띄...  
..                         

In [20]:
df3=pd.concat([df,df2])

In [21]:
print(df3)

                       title   author         isbn13  \
0               달러구트 꿈 백화점 2      이미예  9791165343729   
1                 달러구트 꿈 백화점      이미예  9791165341909   
2                 미드나잇 라이브러리   매트 헤이그  9791191056556   
3   오늘 밤, 세계에서 이 사랑이 사라진다 해도  이치조 미사키  9791191043297   
4                     완전한 행복      정유정  9791167370280   
..                       ...      ...            ...   
87                      동물농장    조지 오웰  9788932909707   
88      사랑하라 한번도 상처받지 않은 것처럼      류시화  9788995501474   
89                       구운몽      김만중  9788937460722   
90                      파리대왕   윌리엄 골딩  9788937460197   
91                   삶의 한가운데   루이제 린저  9788937460289   

                                               review  
0   달러구트 꿈백화점 1권을 읽으신 분들이라면 2권을 손꼽아 기다렸을 것이다. 나 또한...  
1   평소에 책을 너무 안 읽는 것 같아 계속 베스트셀러에 올라와 있기에 주문해 보았습니...  
2   죽음이라는 주제로 어둡게 이야기가 흘러가지는 않을까 걱정했는데 죽음보다는 인생에 대...  
3   우연히 SNS에서 이 책의 줄거리를 읽게 되었습니다. 이런 표지의 감성을 담은 소설...  
4   무서워서 낮에만 읽으려고 했던 나의 결심?다짐?을 반달늪에 쳐박아 버린 소설.

In [22]:
#dataframe to csv
df3.to_csv('bookdata.csv', encoding='utf-8-sig', index=False)